In [1]:
import pandas as pd
import pyodbc

In [2]:
cnxn = pyodbc.connect('Driver={SQL Server};'
                      'Server=EIM-DB-AG40.NORTHGRUM.COM;'
                      'Database=j20032_qn-count;'
                      'Trusted_Connection=yes;')
sql = """SELECT * FROM [j20032_herrfurth_test].[dbo].[TDWH_MetaData_Table]"""
df_meta = pd.read_sql(sql,cnxn)
# getting rid of any trailing whitespace
for col in df_meta.columns:
    try:
        df_meta[col] = df_meta[col].apply(lambda row: row.strip())
    except:
        pass

In [3]:
sql_txt = [x for x in open(r'C:\Users\J20032\Documents\testing_SQL_MetaData.sql','r')]

In [6]:
sql_descs = ''
rec = 0
for y in sql_txt:
    if ('SELECT' in y):
        rec = 1
        sql_descs += y
    elif ('FROM' not in y) and (rec == 1):
        field_nm = y.replace('\\n','').replace('\\t','').strip().replace(',','')
        try:
            desc = df_meta.loc[(df_meta['DFLD_NM']==field_nm) & (df_meta['TXLN_TYP']=='DESC')]['TXLN_TXT'].values[0]
            sql_descs += y.replace('\\n','').replace(',',',\t\t--%s'%desc)
        except IndexError:
            print(field_nm + ' had an index error.')
            sql_descs += y
            pass
    elif (rec == 0):
        sql_descs += y
        pass
    else:
        rec = 0
        sql_descs += y
        pass

QNOT_STAT_TXT had an index error.
PART_DOCU_NO had an index error.
QNOT_ORDR_RTG_NO had an index error.
WCTR_BCTR_ID had an index error.
ORDR_OCAT_CD had an index error.
ORDR_OCAT_TYP had an index error.
ORDR_WBSE_ID had an index error.
Q1.* had an index error.
T.NOTE_TXT had an index error.


In [9]:
with open(r'C:\Users\J20032\Documents\testing_SQL_MetaData_output.sql','w') as txt:
    txt.write(sql_descs)